In [62]:

import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer


In [63]:
nltk.download('wordnet')
ar = pd.read_csv("Amazon_Consumer_Reviews_of_Amazon_Products.csv")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [65]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [66]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
#tokenizer = RegexpTokenizer(r'\w+')
#stop = stopwords.words('english')
#stop = tokenizer.tokenize(stop)
#swn
stop = stopwords.words('english')

In [68]:
# review_list_reg = []
# for review in ar['reviews.text']:
#     review_list_reg.append(tokenizer.tokenize(review))

review_list_reg = []
for review in ar['reviews.text']:
     review_list_reg.append(review)

In [69]:
new_stop = ["i","you","he","she","it","we","they","them","us","him", "me", "her"]
stop_no_pro = []
for word in stop:
    if word in new_stop:
        continue 
    else:
        stop_no_pro.append(word)

In [70]:
def senti_calc(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))




In [71]:
#list = nltk.word_tokenize(review_list_reg[0])

In [72]:
#list

In [73]:
pos_senti_score = []
neg_senti_score = []
for review in review_list_reg:
    #no_punct = tokenizer.tokenize(review)
    tokens = nltk.word_tokenize(review)
    tags = nltk.pos_tag(tokens)
    length = len(tags)
    pos_score = 0
    neg_score = 0
    for word,tag in tags:
        pos_score += senti_calc(word,tag)[0]
        neg_score += senti_calc(word,tag)[1]
    pos_senti_score.append(pos_score/length)
    neg_senti_score.append(neg_score/length)



In [74]:
#print(pos_senti_score[0])
#print(neg_senti_score[0])

In [75]:
review_rating = []
for rating in ar['reviews.rating']:
     review_rating.append(rating)

In [76]:


#classifier = GaussianNB()
classifier = MultinomialNB()


In [77]:
vectorizer = CountVectorizer(stop_words = stop_no_pro)
X = vectorizer.fit_transform(review_list_reg)

C:\Users\JoeKa\anaconda3\envs\cs5050\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['it', 'she', 'you'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [78]:
#vectorizer.get_feature_names_ou
X.shape

(5000, 5285)

In [79]:
#X?

In [80]:
#for x in range(len(pos_senti_score)):
#    np.append(X,pos_senti_score[x],neg_senti_score[
#np.append(X, pos_senti_score, neg_senti_score)
matrix = X.toarray()

In [81]:
pos_senti_score[1]

0.03933974358974359

In [82]:
length = len(pos_senti_score)
for x in range(length):
    pos_senti_score[x] = int(pos_senti_score[x] * 100)

In [83]:
pos_senti_score[0]

4

In [84]:
length = len(neg_senti_score)
for x in range(length):
    neg_senti_score[x] = int(neg_senti_score[x] * 100)

In [85]:
neg_senti_score[0]

5

In [86]:
#matrix?

In [87]:
#sample_arr = np.concatenate((neg_senti_score, pos_senti_score), axis = 0)

In [88]:
#np.hstack((matrix,neg_senti_score))

In [89]:
matrix2 = np.c_[matrix,neg_senti_score]

In [90]:
matrix_f = np.c_[matrix2,pos_senti_score]

In [91]:
matrix_f.shape


(5000, 5287)

In [92]:
matrix_f[4999][5276]

0

In [93]:
#vectorizer.get_params(deep=True)
#Y = vectorizer.inverse_transform(X)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(matrix_f, review_rating)

#X_train, X_test, y_train, y_test = train_test_split(X.toarray(), review_rating)

In [95]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [96]:
y_pred = classifier.predict(X_test)

In [97]:
accuracy = accuracy_score(y_test,y_pred)

In [98]:
accuracy

0.7048

In [99]:
#from sklearn.metrics import accuracy_score,classification_report
#classifier.metrics()

In [100]:
report = classification_report(y_test,y_pred)

C:\Users\JoeKa\anaconda3\envs\cs5050\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
print(report)

              precision    recall  f1-score   support

           1       1.00      0.13      0.24        15
           2       0.00      0.00      0.00         9
           3       0.33      0.02      0.03        56
           4       0.45      0.25      0.32       317
           5       0.75      0.94      0.83       853

    accuracy                           0.70      1250
   macro avg       0.51      0.27      0.28      1250
weighted avg       0.65      0.70      0.65      1250



In [102]:
onestar_class_prob_sorted = classifier.feature_log_prob_[0, :].argsort()[::-1]
twostar_class_prob_sorted = classifier.feature_log_prob_[1, :].argsort()[::-1]
threestar_class_prob_sorted = classifier.feature_log_prob_[2, :].argsort()[::-1]
fourstar_class_prob_sorted = classifier.feature_log_prob_[3, :].argsort()[::-1]
fivestar_class_prob_sorted = classifier.feature_log_prob_[4, :].argsort()[::-1]



In [103]:
print(np.take(vectorizer.get_feature_names(), onestar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), twostar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), threestar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), fourstar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), fivestar_class_prob_sorted[2:10]))

['it' 'amazon' 'bought' 'you' 'kindle' 'tablet' 'week' 'they']
['it' 'you' 'use' 'amazon' 'tablet' 'bought' 'would' 'work']
['it' 'tablet' 'good' 'you' 'kids' 'great' 'use' 'amazon']
['it' 'you' 'tablet' 'great' 'use' 'good' 'amazon' 'like']
['it' 'great' 'love' 'tablet' 'use' 'easy' 'you' 'one']


In [104]:
print(onestar_class_prob_sorted[0:10])
print(twostar_class_prob_sorted[0:10]) 
print(threestar_class_prob_sorted[0:10]) 
print(fourstar_class_prob_sorted[0:10]) 
print(fivestar_class_prob_sorted[0:10])


[5285 5286 2528  316  674 5242 2605 4575 5106 4675]
[5285 5286 2528 5242 4954  316 4575  674 5206 5183]
[5286 5285 2528 4575 2082 5242 2595 2132 4954  316]
[5286 5285 2528 5242 4575 2132 4954 2082  316 2730]
[5286 5285 2528 2132 2818 4575 4954 1531 5242 3191]


In [105]:
print(np.take(vectorizer.get_feature_names(), onestar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), twostar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), threestar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), fourstar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), fivestar_class_prob_sorted[2:50]))


['it' 'amazon' 'bought' 'you' 'kindle' 'tablet' 'week' 'they' 'use' 'junk'
 'last' 'one' 'year' 'get' 'work' 'apps' 'product' 'show' 'them' 'buy'
 'useless' 'device' 'purchased' 'less' 'me' 'constantly' 'best' 'first'
 'back' 'could' 'old' 'fire' 'model' 'would' 'years' 'speakers' 'cloudcam'
 'registered' 'shuts' 'used' 'going' 'lasted' 'days' 'we' 'models' 'good'
 'slow' 'customer']
['it' 'you' 'use' 'amazon' 'tablet' 'bought' 'would' 'work' 'generation'
 'they' 'device' 'apps' 'better' 'quality' 'she' 'sound' 'buy' 'one'
 'kindle' 'nothing' 'much' 'could' 'google' 'echo' 'great' 'screen'
 'returned' 'like' 'ipad' 'hard' 'phone' '4th' 'ads' '5th' '20' 'set'
 'code' 'old' 'every' 'kids' 'time' 'day' 'disappointed' 'needs' 'little'
 'tv' 'frustrating' 'hub']
['it' 'tablet' 'good' 'you' 'kids' 'great' 'use' 'amazon' 'bought' 'ok'
 'apps' 'old' 'time' 'get' 'price' 'echo' 'they' 'one' 'would' 'show'
 'alexa' 'like' 'kindle' 'first' 'store' 'work' 'still' 'play' 'back'
 'google' 'books' 's